# CS 145 COVID Prediction

## Loading Data

First we need to load the data, which is stored in CSV files separated by date

In [40]:
import numpy as np
import pandas as pd
import glob
from sklearn.impute import KNNImputer

In [41]:
# grab filepaths for every csv file
datapath = '../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us/*.csv'
datafiles = np.array(glob.glob(datapath))
print(datafiles[138:142])

['../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us\\08-28-2020.csv'
 '../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us\\08-29-2020.csv'
 '../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us\\08-30-2020.csv'
 '../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us\\08-31-2020.csv']


In [42]:
# init giant df
data = pd.read_csv(datafiles[0])
# read and append each csv to the master df
for i in range(1, 143):
    df = pd.read_csv(datafiles[i])
    data = data.append(df, ignore_index = True)
# only keep 50 states
data.drop(data[data.Province_State.isin(['Diamond Princess', 'District of Columbia', 'Grand Princess', 'Guam', 'Puerto Rico', 'American Samoa', 'Northern Mariana Islands', 'Recovered', 'Virgin Islands'])].index, inplace = True)
data.drop(columns = ['Country_Region', 'Last_Update', 'FIPS', 'UID', 'ISO3'], inplace = True)
data = data.reset_index(drop=True)
data.loc[48:52, ]

,Province_State,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,Testing_Rate,Hospitalization_Rate
48,Wisconsin,44.2685,-89.6165,3341,144,NaN,3197.0,64.565739,39257.0,974.0,4.310087,758.652263,29.152948
49,Wyoming,42.7560,-107.3025,270,0,137.0,270.0,54.299735,5459.0,40.0,0.000000,1097.860188,14.814815
50,Alabama,32.3182,-86.9023,3734,99,NaN,3635.0,79.634933,29182.0,457.0,2.651312,622.363852,12.238886
51,Alaska,61.3707,-152.4044,277,8,85.0,269.0,46.340521,7830.0,32.0,2.888087,1309.914362,11.552347
52,Arizona,33.7298,-111.4312,3705,122,NaN,3583.0,50.901828,43347.0,525.0,3.292848,595.530778,14.170040


## Data Cleaning

In [43]:
data = pd.concat([data, pd.get_dummies(data['Province_State'])], axis = 1)
data.drop(columns = ['Province_State'], inplace = True)
train = 
names = list(data.columns.values)
data.head()

,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,32.3182,-86.9023,3563,93,NaN,3470.0,75.988020,21583.0,437.0,2.610160,...,0,0,0,0,0,0,0,0,0,0
1,61.3707,-152.4044,272,8,66.0,264.0,45.504049,8038.0,31.0,2.941176,...,0,0,0,0,0,0,0,0,0,0
2,33.7298,-111.4312,3542,115,NaN,3427.0,48.662422,42109.0,NaN,3.246753,...,0,0,0,0,0,0,0,0,0,0
3,34.9697,-92.3731,1280,27,367.0,1253.0,49.439423,19722.0,130.0,2.109375,...,0,0,0,0,0,0,0,0,0,0
4,36.1162,-119.6816,22795,640,NaN,22155.0,58.137726,190328.0,5234.0,2.812020,...,0,0,0,0,0,0,0,0,0,0


In [45]:
imputer = KNNImputer(n_neighbors=10, weights="uniform")
data = pd.DataFrame(data = imputer.fit_transform(data), columns = names)
data.head()

,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,32.3182,-86.9023,3563.0,93.0,780.3,3470.0,75.988020,21583.0,437.0,2.610160,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,61.3707,-152.4044,272.0,8.0,66.0,264.0,45.504049,8038.0,31.0,2.941176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,33.7298,-111.4312,3542.0,115.0,1378.2,3427.0,48.662422,42109.0,735.6,3.246753,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,34.9697,-92.3731,1280.0,27.0,367.0,1253.0,49.439423,19722.0,130.0,2.109375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,36.1162,-119.6816,22795.0,640.0,4283.8,22155.0,58.137726,190328.0,5234.0,2.812020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(data.drop(columns = ['Confirmed']), data['Confirmed'])
clf.predict()

['Lat',
 'Long_',
 'Confirmed',
 'Deaths',
 'Recovered',
 'Active',
 'Incident_Rate',
 'People_Tested',
 'People_Hospitalized',
 'Mortality_Rate',
 'Testing_Rate',
 'Hospitalization_Rate',
 'Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']